In [1]:
#Installing requirements
! pip install -r requirements.txt

#Checking Azure CLI. If this fails install Azure CLI here: https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest
!az --version

^C
ERROR: Operation cancelled by user
^C
Traceback (most recent call last):
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/Cellar/azure-cli/2.11.1/libexec/lib/python3.8/site-packages/azure/__init__.py", line 1, in <module>
    ﻿__import__('pkg_resources').declare_namespace(__name__)
  File "/usr/local/Cellar/azure-cli/2.11.1/libexec/lib/python3.8/site-packages/pkg_resources/__init__.py", line 26, in <module>
    import zipfile
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/zipfile.py", line 13, in <module>
    import shutil
  File "/usr/local/Cellar/azure-cli/2.11.1/libexec/lib/python3.8/shutil.py", line 15, in <module

In [23]:
%%bash
#This series of bash commands will create and deploy the Text Analytics for Health Container into an Azure Web App Service

DOCKER_IMAGE_NAME="containerpreview.azurecr.io/microsoft/cognitive-services-healthcare:latest"
subscription_id="" #Example: 2f347497-ec3e-444e-9703-2227ce39ff61
resource_group_name="" #Example: rg-anujbh-biomedicalnlp
resources_location="" #Example: westus2, eastus, etc.
appservice_plan_name="" #Example: P3v2. You'll need at least 4 cores and 8GB RAM
appservice_name="" #Example: biomedicalnlpapp


DOCKER_REGISTRY_SERVER_USERNAME="" #Example: 88c952ab-5221-4e7c-9736-72ba0e393c4c
DOCKER_REGISTRY_SERVER_PASSWORD="" #Example: 82968a04-0b3b-47dd-a121-af97a3f29f17
DOCKER_IMAGE_NAME="containerpreview.azurecr.io/microsoft/cognitive-services-healthcare:latest"

# Login and set subscription
az login
az account set --subscription $subscription_id

# Create text analytics resource
TEXT_ANALYTICS_RESOURCE_NAME="taservice"
az cognitiveservices account create -n $TEXT_ANALYTICS_RESOURCE_NAME -g $resource_group_name --kind TextAnalytics --sku S0 --location westus2

# Get Text Analytics Keys/Info
az cognitiveservices account keys list -n taservice -g rg-anujbh-biomedicalrecognition -o table
az cognitiveservices account show -n taservice -g rg-anujbh-biomedicalrecognition -o jsonc

# Set key and endpoint name
TEXT_ANALYTICS_RESOURCE_API_KEY="" #Example: 18b30e87965c4ea48c690fe748121031. You need to create this separately
TEXT_ANALYTICS_RESOURCE_API_ENDPOINT="" #Example: https://<text_analytics_endpoint>.cognitiveservices.azure.com"

az appservice plan create -n $appservice_plan_name -g $resource_group_name --is-linux -l $resources_location --sku P3V2
az webapp create -g $resource_group_name -p $appservice_plan_name -n $appservice_name -i $DOCKER_IMAGE_NAME -s $DOCKER_REGISTRY_SERVER_USERNAME -w $DOCKER_REGISTRY_SERVER_PASSWORD
az webapp config appsettings set -g $resource_group_name -n $appservice_name --settings Eula=accept Billing=$TEXT_ANALYTICS_RESOURCE_API_ENDPOINT ApiKey=$TEXT_ANALYTICS_RESOURCE_API_KEY


In [142]:
#Once our container is deployed to Azure Web App, we'll need to check to see if it's running. Here's some quick python code to perform the health check every 15 seconds

import requests
import spacy
from spacy import displacy
import json

#TODO: Update this URL with your Web App Service URI
app_service_name = "ta4hwebapp"
endpoint = f"https://{app_service_name}.azurewebsites.net"


In [ ]:
import time

#Check to see if we are retrying
retrying = False

for i in range(0, 2):
    try:
        response = requests.get(endpoint + "/ready")
        if response.status_code == 200:
            print(f"\n===\nHuzzah! Service is up and running at: {endpoint}")
            break
        elif retrying:
            time.sleep(15)
    except Exception as e:
        print("Health check failed:")
        retrying = True


In [148]:
request_data = {
    "documents": [
        {
            "language": "en",
            "id": "1",
            "text": "Subject must have a minimum daily insulin requirement of greater than or equal to 8 units."
        }
    ]
}

# response = requests.post("https://ta4hwebapp.azurewebsites.net/text/analytics/v3.2-preview.1/entities/health",
print(f"{endpoint}/text/analytics/v3.2-preview.1/entities/health")
response = requests.post(f"{endpoint}/text/analytics/v3.2-preview.1/entities/health",
    headers= {
        "Accept": "application/json",
        "Content-Type": "application/json"
    },
    data=json.dumps(request_data))

if response.status_code == 200:
    print(f"\n===\nHuzzah! \n Call to endpoint succeeded: {endpoint}\n")
    print(f"Payload response:\n {response.content[0:100]} \n===\n")



https://ta4hwebapp.azurewebsites.net/text/analytics/v3.2-preview.1/entities/health

===
Huzzah! 
 Call to endpoint succeeded: https://ta4hwebapp.azurewebsites.net

Payload response:
 b'{"documents":[{"id":"1","entities":[{"id":"0","offset":28,"length":5,"text":"daily","category":"Freq' 



In [149]:
colors = {
    "Frequency": "#1e90ff",
    "MedicationName": "#9932cc",
    "RelationalOperator": "#228b22",
    "Dosage": "#ff4500"
}

options = {"ents": ["Frequency","MedicationName","RelationalOperator","Dosage"], "colors": colors}

text = "Subject must have a minimum daily insulin requirement of greater than or equal to 8 units"

entities = response.json()["documents"][0]["entities"]

ent_output = []
for e in entities:
    ent_output.append({"start": e["offset"], "end": e["offset"]+e["length"], "label": e["category"]})

print(str(ent_output) + "\n")

ex = [{"text": text,
       "ents": ents,
       "title": None}]
displacy.render(ex, style="ent", manual=True, )


[{'start': 28, 'end': 33, 'label': 'Frequency'}, {'start': 34, 'end': 41, 'label': 'MedicationName'}, {'start': 57, 'end': 81, 'label': 'RelationalOperator'}, {'start': 82, 'end': 89, 'label': 'Dosage'}]

